In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def transpose_qkv(X, num_heads):
    
    # 输入 `X` 的形状: ( `batch_size`, 查询或者“键－值”对的个数, `num_hiddens` ).
    # 输出 `X` 的形状: ( `batch_size`, 查询或者“键－值”对的个数, `num_heads`,
    # `num_hiddens` / `num_heads` )
    X = X.reshape( X.shape[0], X.shape[1], num_heads, -1 )

    # 输出 `X` 的形状: (`batch_size`, `num_heads`, 查询或者“键－值”对的个数,
    # `num_hiddens` / `num_heads` )
    X = X.permute( 0, 2, 1, 3 )

    # `output` 的形状: (`batch_size` * `num_heads`, 查询或者“键－值”对的个数,
    # `num_hiddens` / `num_heads`)
    return X.reshape( -1, X.shape[2], X.shape[3] )

#
def transpose_output(X, num_heads):
    """逆转 `transpose_qkv` 函数的操作"""
    X = X.reshape(-1, num_heads, X.shape[1], X.shape[2])
    X = X.permute( 0, 2, 1, 3 )
    return X.reshape(X.shape[0], X.shape[1], -1)


In [3]:
class MultiHeadAttention(nn.Module):
    '''多头注意力将输入的特征维度(往往是最后一个维度)使用头(num_heads)拆分，保证最后的输出维度是qkv三种特征维度的的num_head倍，
        d2l解释为多头注意力融合了来自于相同注意力的不同知识，这些知识的不同在于来自相同qkv的不同子空间表示'''
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                 num_heads, dropout, bias=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.attention = d2l.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=bias)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=bias)
        self.W_v = nn.Linear(value_size, num_hiddens, bias=bias)
        self.W_o = nn.Linear(num_hiddens, num_hiddens, bias=bias)

    def forward(self, queries, keys, values, valid_lens):
        # `queries`, `keys`, or `values` 的形状:
        # (`batch_size`, 查询或者“键－值”对的个数, `num_hiddens`)
        # `valid_lens`　的形状:
        # (`batch_size`,) or (`batch_size`, 查询的个数)
        # 经过变换后，输出的 `queries`, `keys`, or `values`　的形状:
        # (`batch_size` * `num_heads`, 查询或者“键－值”对的个数,
        # `num_hiddens` / `num_heads`)
        queries = transpose_qkv( self.W_q(queries), self.num_heads )
        keys = transpose_qkv( self.W_k(keys), self.num_heads )
        values = transpose_qkv( self.W_v(values), self.num_heads )

        if valid_lens is not None:
            # 在轴 0，将第一项（标量或者矢量）复制 `num_heads` 次，
            # 然后如此复制第二项，然后诸如此类。
            valid_lens = torch.repeat_interleave(
                valid_lens, repeats=self.num_heads, dim=0)

        # `output` 的形状: (`batch_size` * `num_heads`, 查询的个数,
        # `num_hiddens` / `num_heads`)
        output = self.attention(queries, keys, values, valid_lens)

        # `output_concat` 的形状: (`batch_size`, 查询的个数, `num_hiddens`)
        output_concat = transpose_output(output, self.num_heads)
        return self.W_o(output_concat)#Wo矩阵是个(num_hiddens , num_hiddens )的形状的矩阵，所以整个多头注意力处理完实际上返回的就是(`batch_size`, 查询的个数, `num_hiddens`)

* 按理来说多头注意力不应该这么做的，原文这样做是要减少参数。实际上我们需要的是Pq = Pk = Pv = Po * num_heads;原代码使用的是Pq /h = Pk /h  = Pv / h = Po的方式
* 如果是自注意力机制，就要保证q,k,v,o四者输出维度保持一致

In [4]:
num_hiddens, num_heads = 100, 5
attention = MultiHeadAttention(num_hiddens, num_hiddens, num_hiddens,
                               num_hiddens, num_heads, 0.5)
attention.train()

MultiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [5]:
batch_size, num_queries, num_kvpairs, valid_lens = 2, 4, 6, torch.tensor([3, 2])
X = torch.ones((batch_size, num_queries, num_hiddens))
Y = torch.ones((batch_size, num_kvpairs, num_hiddens))
attention(X, Y, Y, valid_lens).shape

torch.Size([2, 4, 100])

In [6]:
torch.sigmoid

<function _VariableFunctionsClass.sigmoid>